In [12]:
import pandas as pd
import calendar
from sklearn.metrics import mean_absolute_error
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [9]:
gemma_9b_df = pd.read_csv("AirQuality/RQ1/Dataset/Predictions/gemma2_9b_it_2023_predictions.csv")
gemma_27b_df = pd.read_csv("AirQuality/RQ1/Dataset/Predictions/gemma2_27b_it_2023_predictions.csv")
llama_8b_df = pd.read_csv("AirQuality/RQ1/Dataset/Predictions/llama3_8b_it_2023_predictions.csv")
llama_70b_df = pd.read_csv("AirQuality/RQ1/Dataset/Predictions/llama3_70b_it_2023_predictions.csv")
llama1_70b_df = pd.read_csv("AirQuality/RQ1/Dataset/Predictions/llama3_1_70b_it_2023_predictions.csv")
qwen_32b_df = pd.read_csv("AirQuality/RQ1/Dataset/Predictions/qwen3_32b_2023_predictions.csv")

gemma_9b_aux_df = pd.read_csv("AirQuality/RQ4/Dataset/Aux/gemma2_9b_it_2023_aux.csv")
gemma_27b_aux_df = pd.read_csv("AirQuality/RQ4/Dataset/Aux/gemma2_27b_it_2023_aux.csv")
llama_8b_aux_df = pd.read_csv("AirQuality/RQ4/Dataset/Aux/llama3_8b_it_2023_aux.csv")
llama_70b_aux_df = pd.read_csv("AirQuality/RQ4/Dataset/Aux/llama3_70b_it_2023_aux.csv")
llama1_70b_aux_df = pd.read_csv("AirQuality/RQ4/Dataset/Aux/llama3_1_70b_it_2023_aux.csv")
qwen_32b_aux_df = pd.read_csv("AirQuality/RQ4/Dataset/Aux/qwen3_32b_2023_aux.csv")

gemma_9b_temp_df = pd.read_csv("AirQuality/RQ4/Dataset/Temperature/gemma2_9b_it_2023_temp.csv")
gemma_27b_temp_df = pd.read_csv("AirQuality/RQ4/Dataset/Temperature/gemma2_27b_it_2023_temp.csv")
llama_8b_temp_df = pd.read_csv("AirQuality/RQ4/Dataset/Temperature/llama3_8b_it_2023_temp.csv")
llama_70b_temp_df = pd.read_csv("AirQuality/RQ4/Dataset/Temperature/llama3_70b_it_2023_temp.csv")
llama1_70b_temp_df = pd.read_csv("AirQuality/RQ4/Dataset/Temperature/llama3_1_70b_it_2023_temp.csv")
qwen_32b_temp_df = pd.read_csv("AirQuality/RQ4/Dataset/Temperature/qwen3_32b_2023_temp.csv")

gemma_9b_pop_df = pd.read_csv("AirQuality/RQ4/Dataset/Population/gemma2_9b_it_2023_pop.csv")
gemma_27b_pop_df = pd.read_csv("AirQuality/RQ4/Dataset/Population/gemma2_27b_it_2023_pop.csv")
llama_8b_pop_df = pd.read_csv("AirQuality/RQ4/Dataset/Population/llama3_8b_it_2023_pop.csv")
llama_70b_pop_df = pd.read_csv("AirQuality/RQ4/Dataset/Population/llama3_70b_it_2023_pop.csv")
llama1_70b_pop_df = pd.read_csv("AirQuality/RQ4/Dataset/Population/llama3_1_70b_it_2023_pop.csv")
qwen_32b_pop_df = pd.read_csv("AirQuality/RQ4/Dataset/Population/qwen3_32b_2023_pop.csv")

gemma_9b_ndvi_df = pd.read_csv("AirQuality/RQ4/Dataset/NDVI/gemma2_9b_it_2023_ndvi.csv")
gemma_27b_ndvi_df = pd.read_csv("AirQuality/RQ4/Dataset/NDVI/gemma2_27b_it_2023_ndvi.csv")
llama_8b_ndvi_df = pd.read_csv("AirQuality/RQ4/Dataset/NDVI/llama3_8b_it_2023_ndvi.csv")
llama_70b_ndvi_df = pd.read_csv("AirQuality/RQ4/Dataset/NDVI/llama3_70b_it_2023_ndvi.csv")
llama1_70b_ndvi_df = pd.read_csv("AirQuality/RQ4/Dataset/NDVI/llama3_1_70b_it_2023_ndvi.csv")
qwen_32b_ndvi_df = pd.read_csv("AirQuality/RQ4/Dataset/NDVI/qwen3_32b_2023_ndvi.csv")

In [5]:
ground_truth_df = pd.read_csv("AirQuality/Dataset/Ground_Truth_2023_Final.csv")
ground_truth_df['YearMonth'] = pd.to_datetime(ground_truth_df['YearMonth'])
ground_truth_df['year'] = ground_truth_df['YearMonth'].dt.year
ground_truth_df['month'] = ground_truth_df['YearMonth'].dt.strftime('%B')

ground_truth_df = ground_truth_df.rename(columns={'PM2.5': 'ground_truth_pm25'})

ground_truth_df = ground_truth_df[['city', 'state', 'year', 'month', 'YearMonth', 'ground_truth_pm25']]

In [6]:
ground_truth_df.head()

,city,state,year,month,YearMonth,ground_truth_pm25
0,Agartala,Tripura,2023,January,2023-01-01,196.040103
1,Agartala,Tripura,2023,February,2023-02-01,170.874875
2,Agartala,Tripura,2023,March,2023-03-01,119.490881
3,Agartala,Tripura,2023,April,2023-04-01,94.205356
4,Agartala,Tripura,2023,May,2023-05-01,63.553585


In [18]:
# Standardize text
def preprocess(df):
    df = df.copy()
    df["city"] = df["city"].str.lower().str.strip()
    df["state"] = df["state"].str.lower().str.strip()
    df["month"] = df["month"].str.strip()
    return df

# Prepare ground truth
gt_df = preprocess(ground_truth_df)
gt_df = gt_df[["city", "state", "year", "month", "ground_truth_pm25"]]

# Model list: (model name, no-metadata df, with-metadata df)
model_pairs = [
    ("Gemma2 - 9B", gemma_9b_df, gemma_9b_aux_df),
    ("Gemma2 - 27B", gemma_27b_df, gemma_27b_aux_df),
    ("LLaMA3.1 - 8B", llama_8b_df, llama_8b_aux_df),
    ("LLaMA3.1 - 70B", llama1_70b_df, llama1_70b_aux_df),
    ("LLaMA3.3 - 70B", llama_70b_df, llama_70b_aux_df),
]

In [15]:
# Standardize city/state/month text
def preprocess(df):
    df = df.copy()
    df["city"] = df["city"].str.lower().str.strip()
    df["state"] = df["state"].str.lower().str.strip()
    df["month"] = df["month"].str.strip()
    return df

gt_df = preprocess(ground_truth_df)
gt_df = gt_df[["city", "state", "year", "month", "ground_truth_pm25"]]

# Prepare model pairs
model_pairs = [
    ("Gemma2-9B", gemma_9b_df, gemma_9b_aux_df),
    ("Gemma2-27B", gemma_27b_df, gemma_27b_aux_df),
    ("LLaMA3.1-8B-Instruct", llama_8b_df, llama_8b_aux_df),
    ("LLaMA3.1-70B-Instruct", llama1_70b_df, llama1_70b_aux_df),
    ("LLaMA3.3-70B-Instruct", llama_70b_df, llama_70b_aux_df),
    ("Qwen3-32B", qwen_32b_df, qwen_32b_aux_df)
]

model_variants = {
    "Gemma2-9B": {
        "NoAux": gemma_9b_df,
        "Temp": gemma_9b_temp_df,
        "NDVI": gemma_9b_ndvi_df,
        "Pop": gemma_9b_pop_df,
        "AllAux": gemma_9b_aux_df
    },
    "Gemma2-27B": {
        "NoAux": gemma_27b_df,
        "Temp": gemma_27b_temp_df,
        "NDVI": gemma_27b_ndvi_df,
        "Pop": gemma_27b_pop_df,
        "AllAux": gemma_27b_aux_df
    },
    "LLaMA3.1-8B-Instruct": {
        "NoAux": llama_8b_df,
        "Temp": llama_8b_temp_df,
        "NDVI": llama_8b_ndvi_df,
        "Pop": llama_8b_pop_df,
        "AllAux": llama_8b_aux_df
    },
    "LLaMA3.1-70B-Instruct": {
        "NoAux": llama1_70b_df,
        "Temp": llama1_70b_temp_df,
        "NDVI": llama1_70b_ndvi_df,
        "Pop": llama1_70b_pop_df,
        "AllAux": llama1_70b_aux_df
    },
    "LLaMA3.3-70B-Instruct": {
        "NoAux": llama_70b_df,
        "Temp": llama_70b_temp_df,
        "NDVI": llama_70b_ndvi_df,
        "Pop": llama_70b_pop_df,
        "AllAux": llama_70b_aux_df
    },
    "Qwen3-32B": {
        "NoAux": qwen_32b_df,
        "Temp": qwen_32b_temp_df,
        "NDVI": qwen_32b_ndvi_df,
        "Pop": qwen_32b_pop_df,
        "AllAux": qwen_32b_aux_df
    }
}

### Does adding metadata (e.g., NDVI, temperature, population) improve accuracy?

In [34]:
# results = []

# for model_name, df_no_aux, df_aux in model_pairs:
#     df_no_aux = preprocess(df_no_aux)
#     df_aux = preprocess(df_aux)

#     # Merge with ground truth
#     merged_no_aux = pd.merge(df_no_aux, gt_df, on=["city", "state", "year", "month"], how="inner")
#     merged_aux = pd.merge(df_aux, gt_df, on=["city", "state", "year", "month"], how="inner")

#     # Find common keys
#     common_keys = merged_aux[["city", "state", "year", "month"]].drop_duplicates()
#     merged_no_aux = pd.merge(common_keys, merged_no_aux, on=["city", "state", "year", "month"], how="inner")
#     merged_aux = pd.merge(common_keys, merged_aux, on=["city", "state", "year", "month"], how="inner")

#     # Drop missing values
#     merged_no_aux = merged_no_aux.dropna(subset=["pm2.5", "ground_truth_pm25"])
#     merged_aux = merged_aux.dropna(subset=["pm2.5", "ground_truth_pm25"])

#     # --- Define helper function ---
#     def get_error_stats(df):
#         pred = df["pm2.5"]
#         truth = df["ground_truth_pm25"]
#         error = pred - truth

#         over_mask = error > 0
#         under_mask = error < 0

#         mae_total = mean_absolute_error(truth, pred)
#         mae_over = error[over_mask].abs().mean() if over_mask.any() else 0
#         mae_under = error[under_mask].abs().mean() if under_mask.any() else 0
#         pct_over = 100 * over_mask.sum() / len(df)
#         pct_under = 100 * under_mask.sum() / len(df)

#         return mae_total, mae_over, mae_under, pct_over, pct_under

#     # --- Compute stats ---
#     mae_no_aux, maeover_no_aux, maeunder_no_aux, pctover_no_aux, pctunder_no_aux = get_error_stats(merged_no_aux)
#     mae_aux, maeover_aux, maeunder_aux, pctover_aux, pctunder_aux = get_error_stats(merged_aux)

#     # --- Store results ---
#     results.append({
#         "Model": model_name,

#         "MAE (No Metadata)": round(mae_no_aux, 4),
#         "MAEover (No Metadata)": round(maeover_no_aux, 4),
#         "MAEunder (No Metadata)": round(maeunder_no_aux, 4),
#         "%Over (No Metadata)": round(pctover_no_aux, 2),
#         "%Under (No Metadata)": round(pctunder_no_aux, 2),

#         "MAE (With Metadata)": round(mae_aux, 4),
#         "MAEover (With Metadata)": round(maeover_aux, 4),
#         "MAEunder (With Metadata)": round(maeunder_aux, 4),
#         "%Over (With Metadata)": round(pctover_aux, 2),
#         "%Under (With Metadata)": round(pctunder_aux, 2),

#         "MAE Improvement": round(mae_no_aux - mae_aux, 4)
#     })

# # Convert to DataFrame
# mae_results_df = pd.DataFrame(results)
# mae_results_df

,Model,MAE (No Metadata),MAEover (No Metadata),MAEunder (No Metadata),%Over (No Metadata),%Under (No Metadata),MAE (With Metadata),MAEover (With Metadata),MAEunder (With Metadata),%Over (With Metadata),%Under (With Metadata),MAE Improvement
0,Gemma2 - 9B,28.0184,24.1968,33.2852,57.95,42.05,26.8514,21.2188,34.3709,57.17,42.83,1.1670
1,Gemma2 - 27B,33.1691,34.1674,27.0332,86.01,13.99,31.0131,31.2773,29.7067,83.18,16.82,2.1560
2,LLaMA3.1 - 8B,38.5350,9.7252,42.0121,10.77,89.23,58.6368,0.0000,58.6368,0.00,100.00,-20.1018
3,LLaMA3.1 - 70B,32.3633,36.8056,19.0622,74.96,25.04,28.2293,30.3847,22.4513,72.83,27.17,4.1340
4,LLaMA3.3 - 70B,44.5677,48.5239,19.6678,86.29,13.71,40.4070,43.9975,21.6161,83.96,16.04,4.1607


In [19]:
results = []

for model_name, df_no_aux, df_aux in model_pairs:
    df_no_aux = preprocess(df_no_aux)
    df_aux = preprocess(df_aux)

    # Merge with ground truth
    merged_no_aux = pd.merge(df_no_aux, gt_df, on=["city", "state", "year", "month"], how="inner")
    merged_aux = pd.merge(df_aux, gt_df, on=["city", "state", "year", "month"], how="inner")

    # Find common keys
    common_keys = merged_aux[["city", "state", "year", "month"]].drop_duplicates()
    merged_no_aux = pd.merge(common_keys, merged_no_aux, on=["city", "state", "year", "month"], how="inner")
    merged_aux = pd.merge(common_keys, merged_aux, on=["city", "state", "year", "month"], how="inner")

    # Drop missing values
    merged_no_aux = merged_no_aux.dropna(subset=["pm2.5", "ground_truth_pm25"])
    merged_aux = merged_aux.dropna(subset=["pm2.5", "ground_truth_pm25"])

    # --- Define helper function ---
    def get_error_stats(df):
        pred = df["pm2.5"]
        truth = df["ground_truth_pm25"]
        error = pred - truth

        over_mask = error > 0
        under_mask = error < 0

        mae_total = mean_absolute_error(truth, pred)
        mae_over = error[over_mask].abs().mean() if over_mask.any() else 0
        mae_under = error[under_mask].abs().mean() if under_mask.any() else 0
        pct_over = 100 * over_mask.sum() / len(df)
        pct_under = 100 * under_mask.sum() / len(df)

        return mae_total, mae_over, mae_under, pct_over, pct_under

    # --- Compute stats ---
    mae_no_aux, maeover_no_aux, maeunder_no_aux, pctover_no_aux, pctunder_no_aux = get_error_stats(merged_no_aux)
    mae_aux, maeover_aux, maeunder_aux, pctover_aux, pctunder_aux = get_error_stats(merged_aux)

    # --- Store results ---
    results.append({
        "Model": model_name,

        "MAE (No Metadata)": round(mae_no_aux, 4),
        "MAEover (No Metadata)": round(maeover_no_aux, 4),
        "MAEunder (No Metadata)": round(maeunder_no_aux, 4),
        "%Over (No Metadata)": round(pctover_no_aux, 2),
        "%Under (No Metadata)": round(pctunder_no_aux, 2),

        "MAE (With Metadata)": round(mae_aux, 4),
        "MAEover (With Metadata)": round(maeover_aux, 4),
        "MAEunder (With Metadata)": round(maeunder_aux, 4),
        "%Over (With Metadata)": round(pctover_aux, 2),
        "%Under (With Metadata)": round(pctunder_aux, 2),

        "MAE Improvement": round(mae_no_aux - mae_aux, 4)
    })

# Convert to DataFrame
mae_results_df = pd.DataFrame(results)
mae_results_df

,Model,MAE (No Metadata),MAEover (No Metadata),MAEunder (No Metadata),%Over (No Metadata),%Under (No Metadata),MAE (With Metadata),MAEover (With Metadata),MAEunder (With Metadata),%Over (With Metadata),%Under (With Metadata),MAE Improvement
0,Gemma2 - 9B,28.0184,24.1968,33.2852,57.95,42.05,26.8514,21.2188,34.3709,57.17,42.83,1.1670
1,Gemma2 - 27B,33.1691,34.1674,27.0332,86.01,13.99,31.0131,31.2773,29.7067,83.18,16.82,2.1560
2,LLaMA3.1 - 8B,38.5350,9.7252,42.0121,10.77,89.23,58.6368,0.0000,58.6368,0.00,100.00,-20.1018
3,LLaMA3.1 - 70B,32.3633,36.8056,19.0622,74.96,25.04,28.2293,30.3847,22.4513,72.83,27.17,4.1340
4,LLaMA3.3 - 70B,44.5677,48.5239,19.6678,86.29,13.71,40.4070,43.9975,21.6161,83.96,16.04,4.1607


In [20]:
import numpy as np
import pandas as pd

# === Step 1: Get common subset keys ===
# Create a key for each row to uniquely identify a prediction
def add_key(df):
    df = df.copy()
    df["key"] = df["city"].str.lower() + "_" + df["state"].str.lower() + "_" + df["year"].astype(str) + "_" + df["month"].astype(str)
    return df

# Add key column to all prediction DataFrames
for model_name in model_variants:
    for config_name in model_variants[model_name]:
        model_variants[model_name][config_name] = add_key(model_variants[model_name][config_name])

# Find common keys across all configs of all models
all_keys_sets = []
for model_name in model_variants:
    for config_name, df_pred in model_variants[model_name].items():
        all_keys_sets.append(set(df_pred["key"]))

common_keys = set.intersection(*all_keys_sets)
print(f"Common samples across all configs: {len(common_keys)}")

# === Step 2: Metric calculation on fair subset ===
def preprocess(df):
    # Your existing preprocessing logic if needed
    return df.copy()

def compute_metrics(df_pred, df_gt):
    df_pred = preprocess(df_pred)
    df_pred = df_pred[df_pred["key"].isin(common_keys)]
    df_gt = add_key(df_gt)
    df_gt = df_gt[df_gt["key"].isin(common_keys)]
    
    merged = pd.merge(df_pred, df_gt, on=["key"], how="inner")
    if merged.empty:
        return np.nan, np.nan, np.nan, np.nan, np.nan, 0

    error = merged["pm2.5"] - merged["ground_truth_pm25"]

    over_mask = error > 0
    under_mask = error < 0

    mae = np.mean(np.abs(error))
    mae_over = error[over_mask].abs().mean() if over_mask.any() else 0
    mae_under = error[under_mask].abs().mean() if under_mask.any() else 0
    pct_over = 100 * over_mask.sum() / len(merged)
    pct_under = 100 * under_mask.sum() / len(merged)

    return mae, mae_over, mae_under, pct_over, pct_under, len(merged)

# === Step 3: Run across all model configs ===
records = []
for model_name, configs in model_variants.items():
    for config_name, df_pred in configs.items():
        mae, mae_over, mae_under, pct_over, pct_under, n_samples = compute_metrics(df_pred, gt_df)
        records.append({
            "Model": model_name,
            "Config": config_name,
            "MAE": mae,
            "MAEover": mae_over,
            "MAEunder": mae_under,
            "Over %": pct_over,
            "Under %": pct_under,
            "Samples": n_samples
        })

df_results = pd.DataFrame(records)

# === Step 4: Show comparison ===
metrics = ["MAE", "MAEover", "MAEunder", "Over %", "Under %"]
for metric in metrics:
    table = df_results.pivot(index="Model", columns="Config", values=metric)
    print(f"\n=== {metric} Table (on common samples) ===")
    print(table.round(3))

Common samples across all configs: 1415

=== MAE Table (on common samples) ===
Config                 AllAux    NDVI   NoAux     Pop    Temp
Model                                                        
Gemma2-27B             31.013  30.756  33.169  32.572  30.749
Gemma2-9B              26.851  27.801  28.018  27.424  26.722
LLaMA3.1-70B-Instruct  28.229  35.565  32.363  40.997  28.868
LLaMA3.1-8B-Instruct   58.637  51.428  38.535  48.813  58.256
LLaMA3.3-70B-Instruct  40.407  54.250  44.568  53.980  43.868
Qwen3-32B              33.273  44.214  28.917  31.251  36.326

=== MAEover Table (on common samples) ===
Config                 AllAux    NDVI   NoAux     Pop    Temp
Model                                                        
Gemma2-27B             31.277  32.025  34.167  33.800  31.757
Gemma2-9B              21.219  23.054  24.197  22.897  21.254
LLaMA3.1-70B-Instruct  30.385  39.813  36.806  45.696  32.197
LLaMA3.1-8B-Instruct    0.000   6.650   9.725  13.933   6.412
LLaMA3.3-7

### Does instructing models to abstain (e.g., return NaN) help in low-knowledge settings?

In [25]:
gemma_9b_nan_df = pd.read_csv("AirQuality/RQ1/Dataset/gemma2_9b_it_2023_NaN.csv")
gemma_27b_nan_df = pd.read_csv("AirQuality/RQ1/Dataset/gemma2_27b_it_2023_NaN.csv")
llama_8b_nan_df = pd.read_csv("AirQuality/RQ1/Dataset/llama3_8b_it_2023_NaN.csv")
llama1_70b_nan_df = pd.read_csv("AirQuality/RQ1/Dataset/llama3_1_70b_it_2023_NaN.csv")
llama_70b_nan_df = pd.read_csv("AirQuality/RQ1/Dataset/llama3_70b_it_2023_NaN.csv")
print(gemma_9b_nan_df.head())


       city    state  year     month                 model  pm2.5
0  Agartala  Tripura  2023   January  google/gemma-2-9b-it    NaN
1  Agartala  Tripura  2023  February  google/gemma-2-9b-it    NaN
2  Agartala  Tripura  2023     March  google/gemma-2-9b-it    NaN
3  Agartala  Tripura  2023     April  google/gemma-2-9b-it    NaN
4  Agartala  Tripura  2023       May  google/gemma-2-9b-it    NaN


In [26]:
nan_dfs = {
    "Gemma2 - 9B": gemma_9b_nan_df,
    "Gemma2 - 27B": gemma_27b_nan_df,
    "LLaMA3.1 - 8B": llama_8b_nan_df,
    "LLaMA3.1 - 70B": llama1_70b_nan_df,
    "LlaMA3.3 - 70B": llama_70b_nan_df
}

normal_dfs = {
    "Gemma2 - 9B": gemma_9b_df,
    "Gemma2 - 27B": gemma_27b_df,
    "LLaMA3.1 - 8B": llama_8b_df,
    "LLaMA3.1 - 70B": llama1_70b_df,
    "LlaMA3.3 - 70B": llama_70b_df
}


In [27]:
def standardize_df(df):
    df = df.copy()
    df["city"] = df["city"].str.lower().str.strip()
    df["state"] = df["state"].str.lower().str.strip()
    df["year"] = df["year"].astype(str).str.strip()
    df["month"] = df["month"].str.lower().str.strip()
    return df

def unify_month_format(df):
    df = df.copy()
    month_map = {m.lower(): m for m in calendar.month_name if m}
    df["month"] = df["month"].map(lambda x: month_map.get(x, x))
    return df

def evaluate_model(name, normal_df, nan_df, gt_df):
    print(name)
    # Standardize
    normal_df = unify_month_format(standardize_df(normal_df))
    nan_df = unify_month_format(standardize_df(nan_df))
    gt_df_clean = unify_month_format(standardize_df(gt_df))

    # --- Normal Prompt ---
    normal_merged = pd.merge(normal_df, gt_df_clean, on=["city", "state", "year", "month"], how="inner")
    pct_nan_normal = normal_merged["pm2.5"].isna().mean() * 100  # % NaN before dropna
    normal_merged = normal_merged.dropna(subset=["pm2.5", "ground_truth_pm25"])
    mae_normal = mean_absolute_error(normal_merged["ground_truth_pm25"], normal_merged["pm2.5"]) if len(normal_merged) > 0 else None

    # --- NaN Prompt ---
    nan_merged = pd.merge(nan_df, gt_df_clean, on=["city", "state", "year", "month"], how="inner")
    pct_nan_nan = nan_merged["pm2.5"].isna().mean() * 100  # % NaN before dropna
    nan_merged = nan_merged.dropna(subset=["pm2.5", "ground_truth_pm25"])
    mae_nan = mean_absolute_error(nan_merged["ground_truth_pm25"], nan_merged["pm2.5"]) if len(nan_merged) > 0 else None

    return {
        "Model": name,
        "MAE (Normal Prompt)": round(mae_normal, 4) if mae_normal is not None else None,
        "MAE (NaN Prompt)": round(mae_nan, 4) if mae_nan is not None else None,
        "% NaN (Normal Prompt)": round(pct_nan_normal, 2),
        "% NaN (NaN Prompt)": round(pct_nan_nan, 2),
    }

# Evaluate all models
results = []
for model_name in normal_dfs:
    try:
        result = evaluate_model(
            model_name,
            normal_dfs[model_name],
            nan_dfs[model_name],
            ground_truth_df
        )
        results.append(result)
    except Exception as e:
        print(f"Error in model: {model_name}")
        raise  # re-raise to stop execution and see traceback

# Final result DataFrame
result_df = pd.DataFrame(results)

Gemma2 - 9B
Gemma2 - 27B
LLaMA3.1 - 8B
LLaMA3.1 - 70B
LlaMA3.3 - 70B


In [28]:
result_df

,Model,MAE (Normal Prompt),MAE (NaN Prompt),% NaN (Normal Prompt),% NaN (NaN Prompt)
0,Gemma2 - 9B,27.3261,NaN,0.62,100.00
1,Gemma2 - 27B,34.7241,NaN,0.00,100.00
2,LLaMA3.1 - 8B,37.6088,62.8047,59.12,99.54
3,LLaMA3.1 - 70B,32.6812,46.1367,3.81,94.11
4,LlaMA3.3 - 70B,46.7015,54.5546,0.00,46.14


In [38]:
from sklearn.metrics import mean_absolute_error
import pandas as pd
import calendar

def standardize_df(df):
    df = df.copy()
    df["city"] = df["city"].str.lower().str.strip()
    df["state"] = df["state"].str.lower().str.strip()
    df["year"] = df["year"].astype(str).str.strip()
    df["month"] = df["month"].str.lower().str.strip()
    return df

def unify_month_format(df):
    df = df.copy()
    month_map = {m.lower(): m for m in calendar.month_name if m}
    df["month"] = df["month"].map(lambda x: month_map.get(x, x))
    return df

def compute_errors(df):
    df["abs_error"] = (df["pm2.5"] - df["ground_truth_pm25"]).abs()
    df["over"] = df["pm2.5"] > df["ground_truth_pm25"]
    df["under"] = df["pm2.5"] < df["ground_truth_pm25"]

    mae = df["abs_error"].mean()
    mae_over = df.loc[df["over"], "abs_error"].mean() if df["over"].any() else 0
    mae_under = df.loc[df["under"], "abs_error"].mean() if df["under"].any() else 0
    pct_over = 100 * df["over"].mean()
    pct_under = 100 * df["under"].mean()
    n_samples = len(df)

    return mae, mae_over, mae_under, pct_over, pct_under, n_samples

def evaluate_model(name, normal_df, nan_df, gt_df):
    print(name)
    # Standardize
    normal_df = unify_month_format(standardize_df(normal_df))
    nan_df = unify_month_format(standardize_df(nan_df))
    gt_df_clean = unify_month_format(standardize_df(gt_df))

    # --- Normal Prompt ---
    merged_normal = pd.merge(normal_df, gt_df_clean, on=["city", "state", "year", "month"], how="inner")
    pct_nan_normal = merged_normal["pm2.5"].isna().mean() * 100
    merged_normal = merged_normal.dropna(subset=["pm2.5", "ground_truth_pm25"])
    mae_n, mae_over_n, mae_under_n, pct_over_n, pct_under_n, n_samples_n = compute_errors(merged_normal)

    # --- NaN Prompt ---
    merged_nan = pd.merge(nan_df, gt_df_clean, on=["city", "state", "year", "month"], how="inner")
    pct_nan_nan = merged_nan["pm2.5"].isna().mean() * 100
    merged_nan = merged_nan.dropna(subset=["pm2.5", "ground_truth_pm25"])
    mae_nan, mae_over_nan, mae_under_nan, pct_over_nan, pct_under_nan, n_samples_nan = compute_errors(merged_nan)

    return {
        "Model": name,
        "MAE (Normal)": round(mae_n, 4),
        "MAE Over (Normal)": round(mae_over_n, 4),
        "MAE Under (Normal)": round(mae_under_n, 4),
        "% Over (Normal)": round(pct_over_n, 2),
        "% Under (Normal)": round(pct_under_n, 2),
        "% NaN (Normal)": round(pct_nan_normal, 2),
        "#Samples (Normal)": n_samples_n,

        "MAE (NaN Prompt)": round(mae_nan, 4),
        "MAE Over (NaN Prompt)": round(mae_over_nan, 4),
        "MAE Under (NaN Prompt)": round(mae_under_nan, 4),
        "% Over (NaN Prompt)": round(pct_over_nan, 2),
        "% Under (NaN Prompt)": round(pct_under_nan, 2),
        "% NaN (NaN Prompt)": round(pct_nan_nan, 2),
        "#Samples (NaN Prompt)": n_samples_nan,
    }

In [39]:
results = []
for model_name in normal_dfs:
    try:
        result = evaluate_model(
            model_name,
            normal_dfs[model_name],
            nan_dfs[model_name],
            ground_truth_df
        )
        results.append(result)
    except Exception as e:
        print(f"Error in model: {model_name}")
        raise

final_df = pd.DataFrame(results)
final_df

Gemma2 - 9B
Gemma2 - 27B
LLaMA3.1 - 8B
LLaMA3.1 - 70B
LlaMA3.3 - 70B


,Model,MAE (Normal),MAE Over (Normal),MAE Under (Normal),% Over (Normal),% Under (Normal),% NaN (Normal),#Samples (Normal),MAE (NaN Prompt),MAE Over (NaN Prompt),MAE Under (NaN Prompt),% Over (NaN Prompt),% Under (NaN Prompt),% NaN (NaN Prompt),#Samples (NaN Prompt)
0,Gemma2 - 9B,27.3261,23.9496,32.3280,59.70,40.30,0.62,2397,NaN,0.0000,0.0000,NaN,NaN,100.00,0
1,Gemma2 - 27B,34.7241,35.5347,29.1247,87.35,12.65,0.00,2412,NaN,0.0000,0.0000,NaN,NaN,100.00,0
2,LLaMA3.1 - 8B,37.6088,9.3914,40.2366,8.52,91.48,59.12,986,62.8047,0.0000,62.8047,0.00,100.00,99.54,11
3,LLaMA3.1 - 70B,32.6812,37.0630,19.4142,75.17,24.83,3.81,2320,46.1367,50.6597,21.4662,84.51,15.49,94.11,142
4,LlaMA3.3 - 70B,46.7015,50.1292,21.8161,87.89,12.11,0.00,2412,54.5546,58.5320,25.4126,87.99,12.01,46.14,1299


#### With same set of rows for both (With and without NaN)

In [42]:
def standardize_df(df):
    df = df.copy()
    df["city"] = df["city"].str.lower().str.strip()
    df["state"] = df["state"].str.lower().str.strip()
    df["year"] = df["year"].astype(str).str.strip()
    df["month"] = df["month"].str.lower().str.strip()
    return df

def unify_month_format(df):
    df = df.copy()
    month_map = {m.lower(): m for m in calendar.month_name if m}
    df["month"] = df["month"].map(lambda x: month_map.get(x, x))
    return df

def compute_errors(df):
    df = df.copy() 
    df.loc[:, "abs_error"] = (df["pm2.5"] - df["ground_truth_pm25"]).abs()
    df.loc[:, "over"] = df["pm2.5"] > df["ground_truth_pm25"]
    df.loc[:, "under"] = df["pm2.5"] < df["ground_truth_pm25"]

    mae = df["abs_error"].mean()
    mae_over = df.loc[df["over"], "abs_error"].mean() if df["over"].any() else 0
    mae_under = df.loc[df["under"], "abs_error"].mean() if df["under"].any() else 0
    pct_over = 100 * df["over"].mean()
    pct_under = 100 * df["under"].mean()
    n_samples = len(df)

    return mae, mae_over, mae_under, pct_over, pct_under, n_samples

def evaluate_model(name, normal_df, nan_df, gt_df):
    print(name)
    # Standardize and unify month format
    normal_df = unify_month_format(standardize_df(normal_df))
    nan_df = unify_month_format(standardize_df(nan_df))
    gt_df_clean = unify_month_format(standardize_df(gt_df))

    # Merge with GT
    merged_nan = pd.merge(nan_df, gt_df_clean, on=["city", "state", "year", "month"], how="inner")
    merged_normal = pd.merge(normal_df, gt_df_clean, on=["city", "state", "year", "month"], how="inner")

    # === NaN Prompt Evaluation ===
    pct_nan_nan = merged_nan["pm2.5"].isna().mean() * 100
    merged_nan_valid = merged_nan.dropna(subset=["pm2.5", "ground_truth_pm25"])
    mae_nan, mae_over_nan, mae_under_nan, pct_over_nan, pct_under_nan, n_samples_nan = compute_errors(merged_nan_valid)

    # === Filter Normal to Same Subset as Valid NaN Prompt ===
    valid_keys = merged_nan_valid[["city", "state", "year", "month"]].drop_duplicates()
    merged_normal_filtered = pd.merge(merged_normal, valid_keys, on=["city", "state", "year", "month"], how="inner")

    pct_nan_normal = merged_normal_filtered["pm2.5"].isna().mean() * 100
    merged_normal_valid = merged_normal_filtered.dropna(subset=["pm2.5", "ground_truth_pm25"])
    mae_n, mae_over_n, mae_under_n, pct_over_n, pct_under_n, n_samples_n = compute_errors(merged_normal_valid)

    return {
        "Model": name,
        "MAE (Normal)": round(mae_n, 4),
        "MAE Over (Normal)": round(mae_over_n, 4),
        "MAE Under (Normal)": round(mae_under_n, 4),
        "% Over (Normal)": round(pct_over_n, 2),
        "% Under (Normal)": round(pct_under_n, 2),
        "% NaN (Normal)": round(pct_nan_normal, 2),
        "#Samples (Normal)": n_samples_n,

        "MAE (NaN Prompt)": round(mae_nan, 4),
        "MAE Over (NaN Prompt)": round(mae_over_nan, 4),
        "MAE Under (NaN Prompt)": round(mae_under_nan, 4),
        "% Over (NaN Prompt)": round(pct_over_nan, 2),
        "% Under (NaN Prompt)": round(pct_under_nan, 2),
        "% NaN (NaN Prompt)": round(pct_nan_nan, 2),
        "#Samples (NaN Prompt)": n_samples_nan,
    }

In [43]:
results = []
for model_name in normal_dfs:
    try:
        result = evaluate_model(
            model_name,
            normal_dfs[model_name],
            nan_dfs[model_name],
            ground_truth_df
        )
        results.append(result)
    except Exception as e:
        print(f"Error in model: {model_name}")
        raise

final_df = pd.DataFrame(results)
final_df

Gemma2 - 9B
Gemma2 - 27B
LLaMA3.1 - 8B
LLaMA3.1 - 70B
LlaMA3.3 - 70B


,Model,MAE (Normal),MAE Over (Normal),MAE Under (Normal),% Over (Normal),% Under (Normal),% NaN (Normal),#Samples (Normal),MAE (NaN Prompt),MAE Over (NaN Prompt),MAE Under (NaN Prompt),% Over (NaN Prompt),% Under (NaN Prompt),% NaN (NaN Prompt),#Samples (NaN Prompt)
0,Gemma2 - 9B,NaN,0.0000,0.0000,NaN,NaN,NaN,0,NaN,0.0000,0.0000,NaN,NaN,100.00,0
1,Gemma2 - 27B,NaN,0.0000,0.0000,NaN,NaN,NaN,0,NaN,0.0000,0.0000,NaN,NaN,100.00,0
2,LLaMA3.1 - 8B,68.1320,0.0000,68.1320,0.00,100.00,0.0,11,62.8047,0.0000,62.8047,0.00,100.00,99.54,11
3,LLaMA3.1 - 70B,50.8585,57.3085,19.1460,83.10,16.90,0.0,142,46.1367,50.6597,21.4662,84.51,15.49,94.11,142
4,LlaMA3.3 - 70B,53.4419,56.8716,24.3518,89.45,10.55,0.0,1299,54.5546,58.5320,25.4126,87.99,12.01,46.14,1299
